    Import các thư viện cần thiết

In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from tqdm import tqdm
import csv
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import pandas as pd
import unidecode
from cmath import nan
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler , RobustScaler, StandardScaler
import warnings
import re
import threading
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import os
warnings.filterwarnings("ignore")

    Crawl data

    URL = view-source:https://nhadat24h.net/nha-dat-da-nang/

Khởi tạo đối tượng webdriver và khai báo link URL

In [2]:
#URL = view-source:https://nhadat24h.net/nha-dat-da-nang/
productLinks = []

Lấy tất cả các link của bất động sản tại Đà Nẵng

In [3]:
urlDaNang = "https://nhadat24h.net/nha-dat-da-nang/page{}"
for pagenum in range(1,8):
   try:
      browser = webdriver.Chrome(executable_path="chromedriver")
      browser.get(urlDaNang.format(pagenum))
      tagAs = browser.find_elements(By.CLASS_NAME, "a-title")
      for tagA in tagAs:
         productLinks.append(tagA.get_attribute("href"))
   except:
      continue
   browser.quit()
productLinks

['https://nhadat24h.net/ban-dat-tho-cu-quan-son-tra/ban-nhanh-cap-dat-mt-duong-tri-trach-gia-ban-lo-ca-vai-ti-dong-ID3772381',
 'https://nhadat24h.net/ban-phong-tro-nha-tro-quan-thanh-khe/ban-day-tro-kiet-o-to-6m-nguyen-van-hue-cho-thue-day-khach-gia-tot-nhat-ID3810266',
 'https://nhadat24h.net/ban-dat-tho-cu-quan-thanh-khe/ban-lo-dat-2-mt-quan-thanh-khe-gia-re-nhu-lo-ong-ID3728787',
 'https://nhadat24h.net/ban-dat-tho-cu-quan-lien-chieu/ban-nhanh-lo-dat-mt-nguyen-luong-bang-dat-quoc-lo-gia-bang-dat-du-an-gia-re-ID3731451',
 'https://nhadat24h.net/ban-dat-tho-cu-regal-pavillon/ban-gap-lo-dat-mt-bui-vien-gia-re-bat-ngo-ID3925695',
 'https://nhadat24h.net/ban-biet-thu-duong-dao-xanh-1-7/ban-lo-biet-thu-dao-xanh-khu-vuc-vip-nhat-da-nang-gia-re-nhat-dao-ID3995677',
 'https://nhadat24h.net/ban-biet-thu-kdt-quoc-te-da-phuoc/ban-biet-thu-dep-sang-to-rong-mt-thanh-hai-gan-bien-gia-re-ID3998095',
 'https://nhadat24h.net/ban-dat-tho-cu-quan-cam-le/ban-gap-lo-dat-hiem-con-sot-lai-mt-an-hoa-5-gia-

Lấy các thuộc tính của bất động sản

In [4]:
detailProductsDN = []
for link in productLinks:
    try:
        browser = webdriver.Chrome(executable_path="chromedriver")
        browser.get(link)
        try:
            name = browser.find_element(By.ID, "metaDescription").get_attribute("content")
            detailProduct = {"name": name}
            price = browser.find_element(By.ID, "fromGiatien").get_attribute("value").__add__(" Triệu" if(browser.find_element(By.ID, "dvFromGiatien").get_attribute("value") == "10") else " Tỷ")
            detailProduct["Giá"] = price
        except:
            pass
        try:
            titleValues = browser.find_elements(By.CLASS_NAME, "dv-tb-tsbds")
            for tv in titleValues:
                tvItem = tv.find_elements(By.TAG_NAME, "td")
                detailProduct[tvItem[0].text]= tvItem[1].text
                detailProduct[tvItem[2].text]= tvItem[3].text
        except:
            pass
        try:
            law = browser.find_element(By.TAG_NAME, "i").find_element(By.TAG_NAME, "strong")
            detailProduct["Tình trạng pháp lý"]= law
        except:
            pass
        detailProductsDN.append(detailProduct)
    except:
        print("link")
        continue
    browser.quit()
detailProductsDN

[{'name': 'CHÍNH CHỦ CẦN BÁN NHANH CẶP ĐẤT MT DƯƠNG CHÍ TRẠCH - CẦN TIỀN NÊN BÁN LỖ- Diện tích đất: 130 m2 (10x13) vuông vức- Đất rộng đường lớn ngay gần mặt tiền đường ',
  'Giá': '17 Tỷ',
  'Phòng Ngủ': '',
  'Phòng WC': '',
  'Số tầng': '',
  'Hướng': 'Đông',
  'Đường vào': '5.5 (m)',
  'Mặt tiền': '10 (m)',
  'Chỗ để xe': '4 (Chỗ)',
  'Mã BĐS': '3772381'},
 {'name': 'CẦN BÁN NHANH DÃY TRỌ KIỆT Ô TÔ 6M NGUYỄN VĂN HUỀ - GIÁ RẺ NHƯ MUA ĐẤT- Diện tích đất: 104 m2 - ngang 4,4m nở hậu 4,65m quá đẹp- Dãy trọ kiên cố với 5 phòng (',
  'Giá': '3.7 Tỷ',
  'Phòng Ngủ': '5',
  'Phòng WC': '5',
  'Số tầng': '1 (Tầng)',
  'Hướng': 'Đông bắc',
  'Đường vào': '4.5 (m)',
  'Mặt tiền': '4.4 (m)',
  'Chỗ để xe': '4 (Chỗ)',
  'Mã BĐS': '3810266'},
 {'name': 'CHÍNH CHỦ CẦN BÁN LÔ ĐẤT RỘNG ĐẸP 2 MẶT TIỀN QUẬN THANH KHÊ - GIÁ RẺ- Diện tích đất:\xa0\xa0154.6 m2 (8x20) vuông vức- Lô góc 2 mặt tiền 2 hướng Bắc - Tây nên rất t',
  'Giá': '9.5 Tỷ',
  'Phòng Ngủ': '',
  'Phòng WC': '',
  'Số tầng': '',
  'Hướn

Ghi dữ liệu qua file excel

In [5]:
csv_columns = []
for product in detailProductsDN:
    tempkhac = set(list(product)[0:]) ^ set(csv_columns)
    tempgiong = set(list(product)[0:]) & set(csv_columns)
    csv_columns = tempkhac ^ tempgiong
csv_file = "dataBatDongSanDN24h.csv"
try:
    with open(csv_file, 'w+',encoding= "utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for dataz in detailProductsDN:
            try:
                writer.writerow(dataz)
            except:
                continue
except IOError:
    print("I/O error")